# 第3回講義 宿題

## 課題

今Lessonで学んだことを元に、MNISTのファッション版 (Fashion MNIST、クラス数10) を多層パーセプトロンによって分類してみましょう。

Fashion MNISTの詳細については以下のリンクを参考にしてください。

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール

- 訓練データはx_train、 t_train、テストデータはx_testで与えられます。
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください。
- **下のセルで指定されているx_train、t_train以外の学習データは使わないでください。**
- Pytorchを利用して構いません。
- ただし、**torch.nn.Conv2dのような高レベルのAPIは使用しないで下さい。**具体的には、nn.Parameter, nn.Module, nn.Sequential以外のnn系のAPIです。
- torchvision等で既に実装されているモデルも使用しないで下さい。

### 提出方法

- 2つのファイルを提出していただきます。
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください。
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)。

### 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します。
- 定時に採点しLeader Boardを更新します。(採点スケジュールは別アナウンス）
- 締切後の点数を最終的な評価とします。

### データの読み込み

- この部分は修正しないでください

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import inspect
 
nn_except = ["Module", "Parameter", "Sequential"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0]) 
"""
#学習データ
x_train = np.load('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210422/data/x_train.npy')
t_train = np.load('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210422/data/y_train.npy')
    
#テストデータ
x_test = np.load('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210422/data/x_test.npy')
"""
#学習データ
x_train = np.load('/home/sato.mizuki/deeplearningUT/Lecture03/data/x_train.npy')
t_train = np.load('/home/sato.mizuki/deeplearningUT/Lecture03/data/y_train.npy')
    
#テストデータ
x_test = np.load('/home/sato.mizuki/deeplearningUT/Lecture03/data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 784).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 784).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 多層パーセプトロンの実装

In [2]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [7]:
rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def relu(x):
    x = torch.where(x>0, x, torch.zeros_like(x)) # WRITE ME
    return x


def softmax(x):
    x -= torch.cat([x.max(axis=1, keepdim=True).values]*x.size()[1], dim=1) # WRITE ME
    x_exp = torch.exp(x)
    return x_exp/torch.cat([x_exp.sum(dim=1, keepdim=True)] * x.size()[1], dim=1)

class Dense(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, out_dim, function=lambda x: x):# WRITE ME
        super().__init__()

        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        low=-np.sqrt(6/in_dim),
                        high=np.sqrt(6/in_dim),
                        size=(in_dim, out_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]).astype('float32')))
        self.function = function

    def forward(self, x):  # forwardをoverride
        return self.function(torch.matmul(x, self.W) + self.b)
        

class MLP(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, hid_dim, out_dim): # WRITE ME
        super(MLP, self).__init__()
        self.linear1 = Dense(in_dim, hid_dim)
        self.linear2 = Dense(hid_dim, out_dim)

    def forward(self, x):
        x = relu(self.linear1(x))
        x = softmax(self.linear2(x))
        return x

in_dim = 784
hid_dim = 200
out_dim = 10
lr = 0.001
n_epochs = 2000


mlp = MLP(in_dim, hid_dim, out_dim).to(device) # WRITE ME

optimizer = optim.SGD(mlp.parameters(), lr=lr) # WRITE ME

In [8]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    mlp.train()  # 訓練時には勾配を計算するtrainモードにする
    for x, t in dataloader_train:
        # WRITE ME
        true = t.tolist()

        t_hot = torch.eye(10)[t]
        #テンソルGPUに移動
        x = x.to(device)
        t_hot = t_hot.to(device)
        
        #順伝搬
        y = mlp.forward(x)
        
        #誤差の計算
        loss = -(t_hot*torch.log(y)).sum(axis=1).mean()

        #誤差の逆伝搬
        optimizer.zero_grad()
        loss.backward()

        #パラメータの更新
        optimizer.step()

        # モデルの出力を予測値のスカラーに変換
        pred = y.argmax(1)

        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

    mlp.eval()  # 評価時には勾配を計算しないevalモードにする
    for x, t in dataloader_valid:
        # WRITE ME
        true = t.tolist()

        t_hot = torch.eye(10)[t]
        #テンソルGPUに移動
        x = x.to(device)
        t_hot = t_hot.to(device)
        
        #順伝搬
        y = mlp.forward(x)
        
        #誤差の計算
        loss = -(t_hot*torch.log(y)).sum(axis=1).mean()

        #誤差の逆伝搬
        optimizer.zero_grad()
        loss.backward()

        #パラメータの更新
        optimizer.step()

        # モデルの出力を予測値のスカラーに変換
        pred = y.argmax(1)

        losses_valid.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        valid_num += acc.size()[0]
        valid_true_num += acc.sum().item()

    if epoch%100 == 0:
        print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
            epoch,
            np.mean(losses_train),
            train_true_num/train_num,
            np.mean(losses_valid),
            valid_true_num/valid_num
        ))

EPOCH: 0, Train [Loss: 1.430, Accuracy: 0.577], Valid [Loss: 1.000, Accuracy: 0.696]
EPOCH: 100, Train [Loss: 0.354, Accuracy: 0.879], Valid [Loss: 0.368, Accuracy: 0.870]
EPOCH: 200, Train [Loss: 0.301, Accuracy: 0.896], Valid [Loss: 0.315, Accuracy: 0.888]
EPOCH: 300, Train [Loss: 0.267, Accuracy: 0.907], Valid [Loss: 0.278, Accuracy: 0.902]
EPOCH: 400, Train [Loss: 0.240, Accuracy: 0.917], Valid [Loss: 0.249, Accuracy: 0.912]
EPOCH: 500, Train [Loss: 0.217, Accuracy: 0.925], Valid [Loss: 0.226, Accuracy: 0.921]
EPOCH: 600, Train [Loss: 0.198, Accuracy: 0.934], Valid [Loss: 0.206, Accuracy: 0.926]
EPOCH: 700, Train [Loss: 0.180, Accuracy: 0.940], Valid [Loss: 0.188, Accuracy: 0.934]
EPOCH: 800, Train [Loss: 0.165, Accuracy: 0.947], Valid [Loss: 0.171, Accuracy: 0.942]
EPOCH: 900, Train [Loss: 0.151, Accuracy: 0.952], Valid [Loss: 0.158, Accuracy: 0.949]
EPOCH: 1000, Train [Loss: 0.137, Accuracy: 0.957], Valid [Loss: 0.144, Accuracy: 0.954]
EPOCH: 1100, Train [Loss: 0.126, Accuracy: 0

In [9]:
mlp.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = mlp.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
"""
submission.to_csv('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210422/materials/submission_pred.csv', header=True, index_label='id')
"""
import os
os.makedirs('output', exist_ok=True)
submission.to_csv('output/submission_pred.csv', header=True, index_label='id')